<a href="https://colab.research.google.com/github/dominiquebuford/590-llm-assignment2/blob/main/LLM_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:

# use this command to install open cv2
%pip install opencv-python

# use this command to install PIL
%pip install Pillow

%sudo apt-get install poppler-utils
%pip install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 1s (189 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 121749 

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
%pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
def tokenize_and_chunk(page_text):
  paragraphs = page_text.split('\n\n')
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

  tokenized_paragraphs = [tokenizer(paragraph, return_tensors='pt')['input_ids'].squeeze().tolist() for paragraph in paragraphs]
  filtered_tokens = [_ for _ in tokenized_paragraphs if set(_) != set([101, 102])]
  tokens = filtered_tokens[0]
  print(**tokens)
  chunks = []
  MAX_TOKENS =  200
  tokenCount = 0
  current_chunk = []
  for paragraph in filtered_tokens:
    if tokenCount + len(paragraph) > MAX_TOKENS:
      chunks.append(current_chunk)
      tokenCount = 0
      current_chunk = []
    if len(paragraph) > MAX_TOKENS:
      start_idx = 0
      while start_idx < len(paragraph):
        end_idx = min(start_idx + MAX_TOKENS-2, len(paragraph))
        split_chunk = paragraph[start_idx:end_idx]
        if split_chunk[0]!= 101:      #ensure the chunk has the [CLS] token at the beginning
          split_chunk.insert(0, 101)
        if split_chunk[-1]!= 102:
          split_chunk.append(102)
        chunks.append(split_chunk)
        if end_idx == len(paragraph):
          start_idx = end_idx
        else:
          start_idx = end_idx - 20
      continue
    current_chunk.extend(paragraph)
    tokenCount += len(paragraph)

  if len(current_chunk)>0:
    chunks.append(current_chunk)

  return chunks




In [ ]:
def tokenize_chunk_embed(text):
    #paragraphs = text.split('\n\n')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    max_chunk_size = 512  # Choose your desired chunk size

    # Tokenize each sequence and chunk the tokens
    chunked_sequences = []
    embeddings = []

    #for paragraph in paragraphs:
    tokens_dict = tokenizer(text, return_tensors='pt')
    # Attention mask is automatically added by the tokenizer
    input_ids = tokens_dict['input_ids']

    # Chunk the tokens
    for i in range(0, input_ids.size(1), max_chunk_size-20):
      chunk = input_ids[:, i:i + max_chunk_size]
      chunked_sequences.append(chunk)

      # Process each chunk
    for chunk in chunked_sequences:
      # Forward pass through BERT model
      with torch.no_grad():
        chunk_outputs = model(input_ids=chunk)
        last_hidden_states = chunk_outputs.last_hidden_state

        single_embedding = torch.mean(last_hidden_states, dim=1).view(-1)
        embeddings.append(single_embedding)

    return chunked_sequences, embeddings




In [ ]:
from docx import Document
import pandas as pd
import torch

allChunks = []
allEmbeddings = []
pdf_folder_path = "/content/drive/MyDrive/Masters-2/590-LLM/Homework/Assignment_2/wordDocs/test"
files = os.listdir(pdf_folder_path)
for file in files:
# Open and read the content of the Word document
  path = os.path.join(pdf_folder_path, file)
  doc = Document(path)
  doc_content = ""
  print(len(doc.paragraphs))

  for paragraph in doc.paragraphs:
    fileChunks, fileEmbeddings = tokenize_chunk_embed(paragraph.text)
    allChunks.extend(fileChunks)
    allEmbeddings.extend(fileEmbeddings)



allChunks = [x[0] for x in allChunks]
df_chunk_tokens = pd.DataFrame({'chunks': allChunks,  'embeddings': allEmbeddings})

# Extract the embeddings from the output
df_chunk_tokens.to_csv('/content/tokens_final.csv', index=False)




61


Token indices sequence length is longer than the specified maximum sequence length for this model (620 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (715 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (777 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(allChunks[3])
text = tokenizer.decode(allChunks[23])
print(text)

tensor([ 1996,  5025,  8402,  1997,  2019,  3749,  1012,  1000, 14841, 20444,
        28545,  3090,  2008,  1996,  3945,  2837,  2018,  2042,  1000,  2800,
         2000, 20407,  2013,  1996,  2493,  1010,  1000,  1998,  1999,  2010,
         3193,  1010,  2045,  2001,  2053,  1000,  8050, 28691,  1000,  2090,
         1996,  2837,  1998,  1996, 17694,  1011,  2137,  2554,  1012,  2008,
         5027,  1996,  2554,  5412,  1037,  3116,  2007,  1996,  3945,  2837,
         1010,  1996, 13007,  2094,  2837,  1010, 18523,  6647, 23748,  1998,
         7306,  9957,  3536,  9892,  1012,  2004, 14841, 20444, 28545,  3090,
         1010, 20380,  2015,  2018,  2042,  2081,  1010,  2021,  2045,  2001,
         2053,  2118,  3247,  2664,  1012, 13007,  2094,  7034,  1010,  1000,
         8318,  2003,  5121,  9832,  2008,  1996,  2565,  2071,  3853,  6464,
         2065,  1996,  2472,  2020, 21873,  2000,  1996,  2493,  1012,  1000,
         4826,  1024,  1037,  2472,  2003,  3479,  1012,   102])

In [ ]:
print(len(allChunks))

25


In [ ]:
%pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 5.4 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="17944aa6-d5f5-48f6-843a-aa1e79415857")
index = pc.Index("590-llm-project")
i = 0
for _, row in df_chunk_tokens.iterrows():
  token_chunk = row['chunks']
  embedding = row['embeddings'].numpy().tolist()
  embedding_dict = {}
  embeddingList = []
  embedding_dict['id'] = str(i)
  embedding_dict['values'] = embedding
  embedding_dict['metadata'] = {}
  embedding_dict['metadata']['chunk'] = tokenizer.decode(token_chunk)
  embeddingList.append(embedding_dict)
  index.upsert(vectors=embeddingList)
  i+=1


In [ ]:
vector_example = "How many Black students were enrolled at Duke in 1969?"

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokens_dict = tokenizer(vector_example, return_tensors='pt')
input_ids = tokens_dict['input_ids']
with torch.no_grad():
  chunk_outputs = model(input_ids)
  last_hidden_states = chunk_outputs.last_hidden_state

single_embedding = torch.mean(last_hidden_states, dim=1).view(-1)
finallist = single_embedding.numpy().tolist()
print(",".join(map(str, finallist)))



-0.2774442434310913,-0.15314032137393951,-0.21152669191360474,-0.08726481348276138,0.23514285683631897,0.19983722269535065,0.2850814461708069,0.6772789359092712,-0.5113683938980103,0.16142429411411285,0.4447293281555176,0.035246334969997406,-0.20712542533874512,0.3810097873210907,-0.04219168424606323,0.12101281434297562,0.3331058919429779,-0.17929106950759888,-0.3411107063293457,-0.25397780537605286,-0.14534920454025269,-0.027893899008631706,-0.15765506029129028,0.5617544651031494,0.4859321713447571,-0.11596356332302094,0.19626472890377045,0.1243520975112915,0.0509086474776268,0.06480304151773453,0.12474337220191956,-0.05047542601823807,-0.41771718859672546,0.0937044769525528,-0.4670799672603607,-0.0724208876490593,-0.02533777616918087,0.09832016378641129,0.3169441223144531,0.3668022155761719,-0.942726731300354,-0.29119354486465454,-0.07140481472015381,0.25991758704185486,-0.4216510057449341,-0.24489018321037292,0.6027997136116028,-0.06896768510341644,0.3977562487125397,-0.330832064151

In [ ]:
print(allEmbeddings[21].numpy().tolist())
from sklearn.metrics.pairwise import cosine_similarity
similarityscores= cosine_similarity(single_embedding.numpy().reshape(1, -1), allEmbeddings[23].numpy().reshape(1,-1))
print(similarityscores)

[-0.407054603099823, 0.09991490840911865, 0.12122315168380737, -0.021417012438178062, 0.27619466185569763, -0.10729444772005081, 0.06951545923948288, 0.37169545888900757, -0.04063038155436516, -0.27731388807296753, 0.02180408500134945, -0.0891823098063469, -0.21054336428642273, 0.15984901785850525, -0.06740947812795639, 0.30978941917419434, 0.46039071679115295, -0.09118194133043289, -0.22701114416122437, 0.19456975162029266, 0.0693352073431015, 0.14152589440345764, 0.1185150220990181, 0.60248863697052, 0.42395809292793274, 0.08937353640794754, 0.019101794809103012, -0.1636626273393631, -0.20778045058250427, -0.0860336646437645, 0.643413245677948, -0.0065162512473762035, -0.16075043380260468, -0.29862669110298157, -0.10226297378540039, -0.0676744133234024, -0.005814490839838982, -0.2803872525691986, 0.011356900446116924, 0.14107143878936768, -0.4791681468486786, -0.15875573456287384, -0.30446305871009827, -0.0800161063671112, -0.18453054130077362, -0.17662476003170013, 0.373500883579254

In [ ]:
from transformers import BartTokenizer, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartModel.from_pretrained('facebook/bart-large')

inputs = tokenizer("what was the budget for the program in 1972?", return_tensors="pt")
input_ids = inputs['input_ids']
outputs = model(input_ids)

last_hidden_states = outputs.last_hidden_state

tensor([ 1.0126,  0.1038, -1.7637,  ...,  0.9137,  0.5540,  0.5937],
       grad_fn=<ViewBackward0>)
tensor([ 0.2615, -0.0307,  0.2343,  ...,  0.8356,  0.0234,  0.1768],
       grad_fn=<ViewBackward0>)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'